In [1]:
import db_conn
import pandas as pd
import re

/home/grace/.local/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
conn = db_conn.get_connection()

## 1. 추출된 전체 약물
추출된 전체 약물 중 식별이 가능한 UNIQUE한 약물만을 추려냈습니다.

In [3]:
def get_total_drugs():
    cur = conn.cursor()
    cur.execute("""
    SELECT DISTINCT(TRIM(d.tt_drug)) AS t_d
    FROM
    (SELECT id, tt_drug
    FROM article_table_sentences_m M,
        UNNEST(STRING_TO_ARRAY(t_drug, ' , ')) AS tt_drug
    WHERE (tt_drug='') is false) as d;
    """)
    return cur.fetchall()

In [4]:
drugs = pd.DataFrame(get_total_drugs())

In [5]:
# drugs.to_csv('./drugs.csv', sep=',')

In [6]:
drugs.columns=['name']

In [7]:
drugs['refined'] = ''
drugs.head()

,name,refined
0,rivastigmine,
1,edoxaban,
2,levofloxacin-non-susceptible,
3,procarbazine,
4,bambuterol,


#### 전처리 전 총 약물의 수는 2323개 입니다

In [8]:
drugs.shape

(2323, 2)

### 2. 전처리

#### A. 약물을 포함하고 있는 구절이 추출 된 경우, 약물 외적인 단어들을 제거합니다

## 선생님께서 need_to_refined_hyj.xlsx파일을 통해 약물 이름이 아니라고 지정해주신 목록을 추가하였습니다.

In [9]:
replace_with_space = ['-long-acting', '-treated', '-producing', '-high', '-involved-field', '-based', '-studies', '-non-susceptible',\
                      '-susceptible', '-dependent', '-inducible', '-independent', '-eligible', '-rescue', '-sensitive', '-dense',\
                      'infarction-', '11c]-', 'log[', '[99mtc]', '-avid', '-subcutaneous', '-coated', '-activatable', '-total', '-corrected', \
                      '-predicted', '-gated', '-redo-surgery', '-group', '-assigned', '-allocated', '-reduced', '-combination', \
                      '-standard', '-associated', '-ischemic', '-resistant', '-naive', '-sparing', 'concentration', '[', ']',\
                      'low-molecular-weight-', '-extended', '-meter', '-stimulated', 'subtotal', '-labeled', '-matching',\
                      '-resistance;', '-maintenance', '-rate', '-facilitated', '-replacement', '-to-renin', '-gits', '-treated',\
                      '-right', '-or-2-h', '-treatment', '-boosting', '-resistance', '-continuation', '-rich', '&#8211',\
                      '-impregnated', '-dispensation', '-pemetrexed', '-correctable', '-carry-on', '-oral', '-intravenous', '-bare-metal', '-dose',\
                      '-equivalent', '-induced', '-positive', '-related', '-stimulating', '-confirmed', '-controlled', '(', '-level',\
                      '-releasing', '-reuptake', '-eluting', '-containing', '-binding', 'vaccines', 'vaccine', 'change', 'vsstandard',\
                      
                      # 약물 이름이 아닌 경우
                      'food-borne', 'fast-food', 'food-coping', 'grass-pea-food', 'food-systems', 'food-bolus', 'food-specific', \
                      'food-frequency', 'non-food', 'blood plasma', 'medical air', 'tree pollen', 'grass pollen', 
                      # 추가 약물이름이 아닌 경우
                      'post-food', 'tc-sulfur', 'iodine-to-creatinine', 'protein c2',\
                      'cy5.5-arginine-glycine-aspartate', 'biotin-aacctgaccacagaggagaggc-3', 'n-2-mercaptopropionylglycine', \
                      'biotin-tccctcatccctgccatgt-3', 'animals-l-arg', 'biotin-cacggcacctggcctaaag-3', 'buserelin+cpax14'
                      # 추가 확인 필요
#                       'collagen-sclerotic', 'collagen-septum', 'albumin:creatinine', 'micro-albuminuria','glucose-insulin-potassium', 'oxygen-18', 'albumin<3.3',\
#                       'cyclooxygenase-2', '18f-fluorodeoxyglucose', 'cyclo-oxygenase', 
                     ]

In [10]:
# .....'-the', 'on-', 
_replace_with_space = ['-receptor'
                      'post-','-dose-dense', '–placebo', \
                      '-inhibiting', '-containing', '-related', '-like', '-boosted', '-binding', '8 × ', '-only', \
                      '-pulse', '-positive', '-confirmed', '-controlled', '-releasing', '-lowering', '≥110', \
                      '-independent', '≥5', 'secreting', 'mothers', 'bothersome', "mothers'", 'senegal']
# '-dependent', , 

In [ ]:
rep = dict((re.escape(k), ' ') for k in replace_with_space)
pattern = re.compile("|".join(rep.keys()))

for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if refined.strip() == '':
        drugs.drop(ind, inplace=True)
    elif row['name'].strip() != refined.strip():
        drugs.loc[ind]['refined'] = refined
        print(row['name'])
        print(drugs.loc[ind]['refined'])
    else:
        print(row['name'])
    print('===============================')

drugs = drugs.reset_index(drop=True)

#### 전처리와 약물이 아닌 레코드를 제거하면 2301개가 남습니다

In [12]:
drugs.shape

(2301, 2)

## 선생님께 확인 요청을 드리고 싶은 부분

## 1. albumin:creatinine 이형태로 표기된 애들은 약제가 아니라고 해주셨는데, 그럼 둘을 포함하고 있는 전체를 제외해도 될까요?

## 아니면 albumin, creatinine 둘 중 하나라도 포함하고 있다면 제외해도 될까요?

In [13]:
drugs[drugs.name.str.find('albumin')>=0]

,name,refined
45,albumin,
53,albumin:creatinine,
119,albumin<3.2,
154,microalbumin,
163,albumincreatinine,
303,micro-albuminuria,
382,macroalbuminuria,
633,normoalbuminuric,
660,albumin-to,
715,albumin-adjusted,


In [14]:
drugs[drugs.name.str.find('creatinine')>=0]

,name,refined
53,albumin:creatinine,
163,albumincreatinine,
981,albumin–creatinine,
1136,albumin-to-creatinine,
1547,albumin-creatinine,
1833,protein creatinine,


In [15]:
drugs[(drugs.name.str.find('creatinine')>=0) & (drugs.name.str.find('albumin')>=0)]

,name,refined
53,albumin:creatinine,
163,albumincreatinine,
981,albumin–creatinine,
1136,albumin-to-creatinine,
1547,albumin-creatinine,


## 2. collagen을 포함한 경우 제외해도 될까요?

In [16]:
drugs[drugs.name.str.find('collagen')>=0]

,name,refined
41,collagenase,
160,collagen-induced,collagen
208,collagen,
253,collagen-right,collagen
444,collagen-sclerotic,
447,collagen-septum,
781,collagen:,
1360,collagen.,
1442,pah–collagen,
2011,procollagen,


## 3. blood을 포함한 경우 제외해도 될까요?

In [17]:
drugs[drugs.name.str.find('blood')>=0]

,name,refined
846,blood-oxygen-level-dependent,blood-oxygen
1099,blood-glucose-lowering,
1252,bloodglucose,
1401,blood-glucose,


## 4. glucose를 포함한 경우 제외해도 될까요?

In [18]:
drugs[drugs.name.str.find('glucose')>=0]

,name,refined
147,glucose-dependent,glucose
212,-fluoro-2-deoxy-d-glucose,
218,2-18f-fluoro-2-deoxy-d-glucose-avid,2-18f-fluoro-2-deoxy-d-glucose
295,glucose-or-2-h,glucose
586,glucose],glucose
592,glucose-6-phosphate,
685,glucoseauc,
801,glucose-containing,glucose
834,glucose-confirmed,glucose
875,glucose:,


## 5. oxygen을 포함한 경우 제외해도 될까요?

In [19]:
drugs[drugs.name.str.find('oxygen')>=0]

,name,refined
28,oxygenator,
82,oxygen-dependent,oxygen
191,preoxygenate,
361,oxygen-treatment,oxygen
598,oxygen,
846,blood-oxygen-level-dependent,blood-oxygen
913,oxygen.,
1395,oxygenation,
1465,cyclo-oxygenase,
1534,oxygen-18,


## 6. nitrogen을 포함한 경우 제외해도 될까요?

In [20]:
drugs[drugs.name.str.find('nitrogen')>=0]

,name,refined
116,nitrogen,
1229,nitrogen-13,
1637,nitrogen-containing,nitrogen


## 7. thrombin을 포함한 경우 제외해도 될까요?

In [21]:
drugs[drugs.name.str.find('thrombin')>=0]

,name,refined
278,thrombin.,
609,thrombin-induced,thrombin
663,thrombin-activatable,thrombin
950,antithrombin,
990,ma-thrombin,
1220,thrombin-antithrombin,
1275,thrombin-stimulated,thrombin
1343,thrombin,
1426,antithrombin iii,
1638,thrombin&#8211,thrombin


## 8. fetoprotein을 포함한 경우 제외해도 될까요?

In [22]:
drugs[drugs.name.str.find('fetoprotein')>=0]

,name,refined
983,α-fetoprotein concentration,α-fetoprotein
2212,α-fetoprotein,


## 9. renin을 포함한 경우 제외해도 될까요?

In [23]:
drugs[drugs.name.str.find('renin')>=0]

,name,refined
219,aldosterone-to-renin,aldosterone
1674,renin-angiotensin-aldosterone,
1918,renin–angiotensin–aldosterone,


#### B. 약-부작용 관계를 나타는 것이 아닌 경우, 지웁니다

### 선생님께서 피드백 주신데로 non은 제외하였습니다.
### 선생님께서 피드백 주신데로 약물명 전처리에서 non-제거 & resistant, refractory, pre-, -insensitive, -prev, lipoprotein, untreated, anti-, -receptor 추가하였습니다.

In [24]:
non_drug_ad_rel = ['pseudo', 'placebo', 'negative', 'free', 'withdrawal', 'blocking', 'deficient', 'inhibiting',
                  '-prev', '-insensitive', 'resistant', 'refractory', 'pre-', 'lipoprotein', '-untreated', 'anti-',
                  '-receptor', '-tolerance', '-intolerant']
#'non-'

rep = dict((re.escape(k), ' ') for k in non_drug_ad_rel)
pattern = re.compile("|".join(rep.keys()))

In [25]:
for ind, row in drugs.iterrows():
    refined = pattern.sub(lambda m: rep[re.escape(m.group(0))], row['name'])
    if row['name'].strip() != refined.strip():
        print(row['name'])
        drugs.drop(ind, inplace=True)
        print('===============================')

drugs = drugs.reset_index(drop=True)

pseudo-cholinesterase
low-density-lipoprotein c
high-density-lipoprotein cholesterol
heparin-placebo
donepezilplacebo
salmeterolplacebo
anti-aldosterone
rifampicin-resistant
rituximab-refractory
progesterone-receptor
low-density-lipoprotein cholesterol
pre-sildenafil
cocaine-negative
tetracycline-resistant
pre-abciximab
iodine-deficient
oseltamivir-resistant
pre-carvedilol
erythromycin-resistant
pre-sirolimus-eluting
lipoprotein cholest
nicotine-free
pre-docetaxel
pre-ivermectin
salmeterol+placebo
buserelin+placebo
placebo+prednisone
tuberculin-negative
ganciclovir-resistant
carvedilol-untreated
pre-mibefradil
enalapril-prev
ceftriaxone-resistant
clindamycin-resistant
interferon-intolerant
cisplatin-refractory
pre-terlipressin
adenosine-insensitive
lipoprotein cholesterolmg
pre-albumin
placebodocetaxel
anti-exenatide
methylprednisolone-placebo
bevacizumab-free
pre-propafenone
high-density-lipoprotein c
anti-interferon
anti-evolocumab
pre-dobutamine
placebo-clopidogrel
cefotaxime-resist

#### 약-부작용 관계를 나타는 것이 아닌 레코드를 제거하면 2237 남습니다

In [27]:
drugs.shape

(2237, 2)